## Processing ARF data

In [273]:
import os
import struct
import datetime
# from skfda import FDataGrid
# from skfda.preprocessing.dim_reduction import FPCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import struct
import matplotlib.pyplot as plt
import torch
from scipy.ndimage import gaussian_filter1d
import torch.nn as nn
import splitfolders

# ARCHIVE

In [274]:
abr_path = "/Users/leahashebir/Downloads/Manor_Practicum/manor_lauren/abr_data/Samp8 A1 baseline"
for file in os.listdir(abr_path):
    path = os.path.abspath(os.path.join(abr_path,file))
    data = arfread1(path, RP=False)
    
    wave_data = []
    for group in range(data['RecHead']['ngrps']):
        #this example has only 1 group
        print('group #', group)
        for rec in range(data['groups'][group]['nrecs']):
            # this one group has many recordings (different dBs, freqs)

            # freq = data['groups'][group]['recs'][rec]['Var2']
            freq = '?'
            dB = data['groups'][group]['recs'][rec]['Var1']
            wave_data.append(data['groups'][0]['recs'][rec]['data'])

            gain = data['groups'][group]['recs'][rec]['gain'] # this might be where that 20x multiplier comes from? double check with elena
            total_t = data['groups'][group]['recs'][rec]['dur_ms']

            print('recording #',
                    rec,
                    'duration (ms)',
                    total_t,
                    'gain',
                    gain,
                    'Freq?',
                    freq,
                    'dB?',
                    dB)

group # 0
recording # 0 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 4000.0
recording # 1 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 4000.0
recording # 2 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 4000.0
recording # 3 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 4000.0
recording # 4 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 4000.0
recording # 5 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 4000.0
recording # 6 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 4000.0
recording # 7 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 4000.0
recording # 8 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 4000.0
recording # 9 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 4000.0
recording # 10 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 8000.0
recording # 11 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 8000.0
recording # 12 duration (ms) 9.994239807128906 gain 20.0 Freq? ? dB? 8000.0
recording # 

IndexError: list index out of range

In [ ]:
# Original logic

abr_path = "/Users/leahashebir/Downloads/Manor_Practicum/manor_lauren/abr_data"
abr_directory = os.listdir(abr_path)

all_abr = []

for file in abr_directory:
    path = os.path.abspath(os.path.join(abr_path,file))
    data = arfread(path) #Believe it is RZ since RP doesn't work

    abrs = []
    for group in data['groups']: # there are multiple groups under each file, varies per file
        for rec in group['recs']:
            wave_cols = list(enumerate(rec['data']))
            wave_data = {f'{i}':v*1e6 for i, v in wave_cols} 
            abrs.append(wave_data)
        all_abr.append(abrs)

# len(all_abr[1])

In [ ]:
def interpolate_and_smooth(final, target_length=244):
    if len(final) > target_length:
        new_points = np.linspace(0, len(final), target_length + 2)
        interpolated_values = np.interp(new_points, np.arange(len(final)), final)
        final = np.array(interpolated_values[:target_length], dtype=float)
    elif len(final) < target_length:
        original_indices = np.arange(len(final))
        target_indices = np.linspace(0, len(final) - 1, target_length)
        cs = CubicSpline(original_indices, final)
        final = cs(target_indices)
    return final

def calculate_and_plot_wave(df, freq, db, color, threshold=None):
    khz = df[(df['Freq(Hz)'] == freq) & (df[db_column] == db)]
    if not khz.empty:
        index = khz.index.values[0]
        final = df.loc[index, '0':].dropna()
        final = pd.to_numeric(final, errors='coerce').dropna()

        target = int(244 * (time_scale / 10))
        
        y_values = interpolate_and_smooth(final, target)  # Original y-values for plotting
        sampling_rate = len(y_values) / time_scale  # Assuming 10 ms duration for 244 points

        x_values = np.linspace(0, len(y_values) / sampling_rate, len(y_values))

        y_values_for_peak_finding = interpolate_and_smooth(final[:244])
        if units == 'Nanovolts':
            y_values /= 1000

        y_values_for_peak_finding *= multiply_y_factor

        highest_peaks, relevant_troughs = peak_finding(y_values_for_peak_finding)

        return x_values, y_values, highest_peaks, relevant_troughs
    return None, None, None, None

def calculate_hearing_threshold(df, freq, baseline_level=100, multiply_y_factor=1):
    db_column = 'Level(dB)' if level else 'PostAtten(dB)'

    thresholding_model = load_model('./models/abr_cnn_aug_norm_opt.keras')
    thresholding_model.steps_per_execution = 1
    
    # Filter DataFrame to include only data for the specified frequency
    df_filtered = df[df['Freq(Hz)'] == freq]

    # Get unique dB levels for the filtered DataFrame
    db_levels = sorted(df_filtered[db_column].unique()) if db_column == 'Level(dB)' else sorted(df_filtered[db_column].unique(), reverse=True)
    
    lowest_db = None
    waves = []
    previous_prediction = None  # Initialize previous prediction to track consecutive `1`s

    for db in db_levels:
        khz = df_filtered[df_filtered[db_column] == np.abs(db)]
        if not khz.empty:
            index = khz.index.values[-1]
            final = df_filtered.loc[index, '0':].dropna()
            final = pd.to_numeric(final, errors='coerce')
            final = np.array(final, dtype=np.float64)
            final = interpolate_and_smooth(final)

            if units == 'Nanovolts':
                final /= 1000

            waves.append(final)
    
    waves = np.array(waves)
    flattened_data = waves.flatten().reshape(-1, 1)
    scaler = StandardScaler()
    scaled_flattened_data = scaler.fit_transform(flattened_data).reshape(waves.shape)
    waves = np.expand_dims(scaled_flattened_data, axis=2)
    
    # Perform prediction
    prediction = thresholding_model.predict(waves)
    y_pred = (prediction > 0.5).astype(int).flatten()

    for p, d in zip(y_pred, db_levels):
        if p == 0:
            if db_column == 'PostAtten(dB)':
                calibration_key = (df.name, freq)  # Assuming df has an attribute 'name' set to the file name
                if calibration_key in calibration_levels:
                    calibration_level = calibration_levels[calibration_key]
                else:
                    calibration_level = 0
                lowest_db = baseline_level - (d + calibration_level)
            else:
                lowest_db = d
            previous_prediction = p  # Update previous prediction
        else:
            if previous_prediction == 1:
                break  # Break if two consecutive `1`s are encountered
            if db_column == 'PostAtten(dB)':
                calibration_key = (df.name, freq)
                if calibration_key in calibration_levels:
                    calibration_level = calibration_levels[calibration_key]
                else:
                    calibration_level = 0
                lowest_db = baseline_level - (d + calibration_level)
            else:
                lowest_db = d
            previous_prediction = p  # Update previous prediction
    
    return lowest_db

In [ ]:
# Normalization is included in teh peak finding code...

sc = StandardScaler()
scaled = {}
all_scaled = {}

for data in all_abr:
    collect_scaled = [] #entire file contents
    for group in data:
        # print(group)
        to_be_scaled = list(group.values()) #per group
        # print(to_be_scaled)
        to_be_scaled_array = np.array(to_be_scaled).reshape(-1,1)
        scaled_per_group = sc.fit_transform(to_be_scaled_array)
        collect_scaled.append(scaled_per_group)
        # print(len(collect_scaled[0]))
    for i,v in enumerate(collect_scaled):
        all_scaled[i] = v
        
    
all_scaled

{0: array([[ 0.23478679],
        [ 0.25026202],
        [ 0.25729211],
        [ 0.26100398],
        [ 0.26489187],
        [ 0.26675353],
        [ 0.26606828],
        [ 0.26505867],
        [ 0.26550708],
        [ 0.26729428],
        [ 0.26732707],
        [ 0.26335623],
        [ 0.25657103],
        [ 0.25165754],
        [ 0.25315466],
        [ 0.26194761],
        [ 0.28018201],
        [ 0.30635082],
        [ 0.32995362],
        [ 0.33812208],
        [ 0.32452506],
        [ 0.29680737],
        [ 0.26831197],
        [ 0.24753983],
        [ 0.23343725],
        [ 0.22014786],
        [ 0.20474104],
        [ 0.18919983],
        [ 0.17701441],
        [ 0.16765739],
        [ 0.1578908 ],
        [ 0.14817846],
        [ 0.14305614],
        [ 0.14671673],
        [ 0.1594742 ],
        [ 0.18138545],
        [ 0.21017231],
        [ 0.23894738],
        [ 0.26139797],
        [ 0.27128889],
        [ 0.26437307],
        [ 0.23915835],
        [ 0.2027348 ],
        

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class GraphToImageAutoencoder(nn.Module):
    def __init__(self, input_size, img_size):
        super(GraphToImageAutoencoder, self).__init__()
        # Encoder: Graph data → latent space
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        # Decoder: Latent space → Image
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, img_size * img_size),
            nn.Sigmoid()  # Output in range [0, 1]
        )
        self.img_size = img_size

    def forward(self, x):
        latent = self.encoder(x)
        decoded = self.decoder(latent)
        return decoded.view(-1, 1, self.img_size, self.img_size)

# Initialize model
input_size = 10  # Graph features
img_size = 64  # Output image size (64x64)
model = GraphToImageAutoencoder(input_size, img_size)

# Loss: Mean Squared Error for pixel-wise differences
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Concatenate

image_input = Input(shape=(64, 64, 3))  # Shape of the image input
numerical_input = Input(shape=(10,))  # Shape of the numerical data

# Example: Define separate encoders for each modality
image_encoder = Flatten()(image_input)  # Flatten image to 1D vector
image_encoder = Dense(128, activation='relu')(image_encoder)

numerical_encoder = Dense(128, activation='relu')(numerical_input)

# Concatenate the encoded representations
merged = Concatenate()([image_encoder, numerical_encoder])

decoder = Dense(128, activation='relu')(merged)
decoder = Dense(64, activation='relu')(decoder)
output = Dense(64 * 64 * 3, activation='sigmoid')(decoder)  # Reconstruct the image

output = Reshape((64, 64, 3))(output)

# Build and compile the model
model = Model(inputs=[image_input, numerical_input], outputs=output)
model.compile(optimizer='adam', loss='mse')

# ARF Readers

In [ ]:
def get_str(data):
    # return string up until null character only
    ind = data.find(b'\x00')
    if ind > 0:
        data = data[:ind]
    return data.decode('utf-8')

def arfread(PATH, **kwargs):
    # defaults
    PLOT = kwargs.get('PLOT', False)
    RP = kwargs.get('RP', False)
    
    isRZ = not RP
    
    data = {'RecHead': {}, 'groups': []}

    # open file
    with open(PATH, 'rb') as fid:
        # open RecHead data
        data['RecHead']['ftype'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['ngrps'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['nrecs'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['grpseek'] = struct.unpack('200i', fid.read(4*200))
        data['RecHead']['recseek'] = struct.unpack('2000i', fid.read(4*2000))
        data['RecHead']['file_ptr'] = struct.unpack('i', fid.read(4))[0]

        data['groups'] = []
        bFirstPass = True
        for x in range(data['RecHead']['ngrps']):
            # jump to the group location in the file
            fid.seek(data['RecHead']['grpseek'][x], 0)

            # open the group
            data['groups'].append({
                'grpn': struct.unpack('h', fid.read(2))[0],
                'frecn': struct.unpack('h', fid.read(2))[0],
                'nrecs': struct.unpack('h', fid.read(2))[0],
                'ID': get_str(fid.read(16)),
                'ref1': get_str(fid.read(16)),
                'ref2': get_str(fid.read(16)),
                'memo': get_str(fid.read(50)),
            })

            # read temporary timestamp
            if bFirstPass:
                if isRZ:
                    ttt = struct.unpack('q', fid.read(8))[0]
                    fid.seek(-8, 1)
                    data['fileType'] = 'BioSigRZ'
                else:
                    ttt = struct.unpack('I', fid.read(4))[0]
                    fid.seek(-4, 1)
                    data['fileType'] = 'BioSigRP'
                data['fileTime'] = datetime.datetime.utcfromtimestamp(ttt/86400 + datetime.datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S')
                bFirstPass = False

            if isRZ:
                grp_t_format = 'q'
                beg_t_format = 'q'
                end_t_format = 'q'
                read_size = 8
            else:
                grp_t_format = 'I'
                beg_t_format = 'I'
                end_t_format = 'I'
                read_size = 4

            data['groups'][x]['beg_t'] = struct.unpack(beg_t_format, fid.read(read_size))[0]
            data['groups'][x]['end_t'] = struct.unpack(end_t_format, fid.read(read_size))[0]

            data['groups'][x].update({
                'sgfname1': get_str(fid.read(100)),
                'sgfname2': get_str(fid.read(100)),
                'VarName1': get_str(fid.read(15)),
                'VarName2': get_str(fid.read(15)),
                'VarName3': get_str(fid.read(15)),
                'VarName4': get_str(fid.read(15)),
                'VarName5': get_str(fid.read(15)),
                'VarName6': get_str(fid.read(15)),
                'VarName7': get_str(fid.read(15)),
                'VarName8': get_str(fid.read(15)),
                'VarName9': get_str(fid.read(15)),
                'VarName10': get_str(fid.read(15)),
                'VarUnit1': get_str(fid.read(5)),
                'VarUnit2': get_str(fid.read(5)),
                'VarUnit3': get_str(fid.read(5)),
                'VarUnit4': get_str(fid.read(5)),
                'VarUnit5': get_str(fid.read(5)),
                'VarUnit6': get_str(fid.read(5)),
                'VarUnit7': get_str(fid.read(5)),
                'VarUnit8': get_str(fid.read(5)),
                'VarUnit9': get_str(fid.read(5)),
                'VarUnit10': get_str(fid.read(5)),
                'SampPer_us': struct.unpack('f', fid.read(4))[0],
                'cc_t': struct.unpack('i', fid.read(4))[0],
                'version': struct.unpack('h', fid.read(2))[0],
                'postproc': struct.unpack('i', fid.read(4))[0],
                'dump': get_str(fid.read(92)),
                'recs': [],
            })

            for i in range(data['groups'][x]['nrecs']):
                record_data = {
                        'recn': struct.unpack('h', fid.read(2))[0],
                        'grpid': struct.unpack('h', fid.read(2))[0],
                        'grp_t': struct.unpack(grp_t_format, fid.read(read_size))[0],
                        #'grp_d': datetime.utcfromtimestamp(data['groups'][x]['recs'][i]['grp_t']/86400 + datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S'),
                        'newgrp': struct.unpack('h', fid.read(2))[0],
                        'sgi': struct.unpack('h', fid.read(2))[0],
                        'chan': struct.unpack('B', fid.read(1))[0],
                        'rtype': get_str(fid.read(1)),
                        'npts': struct.unpack('H' if isRZ else 'h', fid.read(2))[0],
                        'osdel': struct.unpack('f', fid.read(4))[0],
                        'dur_ms': struct.unpack('f', fid.read(4))[0],
                        'SampPer_us': struct.unpack('f', fid.read(4))[0],
                        'artthresh': struct.unpack('f', fid.read(4))[0],
                        'gain': struct.unpack('f', fid.read(4))[0],
                        'accouple': struct.unpack('h', fid.read(2))[0],
                        'navgs': struct.unpack('h', fid.read(2))[0],
                        'narts': struct.unpack('h', fid.read(2))[0],
                        'beg_t': struct.unpack(beg_t_format, fid.read(read_size))[0],
                        'end_t': struct.unpack(end_t_format, fid.read(read_size))[0],
                        'Var1': struct.unpack('f', fid.read(4))[0],
                        'Var2': struct.unpack('f', fid.read(4))[0],
                        'Var3': struct.unpack('f', fid.read(4))[0],
                        'Var4': struct.unpack('f', fid.read(4))[0],
                        'Var5': struct.unpack('f', fid.read(4))[0],
                        'Var6': struct.unpack('f', fid.read(4))[0],
                        'Var7': struct.unpack('f', fid.read(4))[0],
                        'Var8': struct.unpack('f', fid.read(4))[0],
                        'Var9': struct.unpack('f', fid.read(4))[0],
                        'Var10': struct.unpack('f', fid.read(4))[0],
                        'data': [] #list(struct.unpack(f'{data["groups"][x]["recs"][i]["npts"]}f', fid.read(4*data['groups'][x]['recs'][i]['npts'])))
                    }
                
                # skip all 10 cursors placeholders
                fid.seek(36*10, 1)
                record_data['data'] = list(struct.unpack(f'{record_data["npts"]}f', fid.read(4*record_data['npts'])))

                record_data['grp_d'] = datetime.datetime.utcfromtimestamp(record_data['grp_t'] / 86400 + datetime.datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S')

                data['groups'][x]['recs'].append(record_data)

            if PLOT:
                import matplotlib.pyplot as plt

                # determine reasonable spacing between plots
                d = [x['data'] for x in data['groups'][x]['recs']]
                plot_offset = max(max(map(abs, [item for sublist in d for item in sublist]))) * 1.2

                plt.figure()

                for i in range(data['groups'][x]['nrecs']):
                    plt.plot([item - plot_offset * i for item in data['groups'][x]['recs'][i]['data']])
                    plt.hold(True)

                plt.title(f'Group {data["groups"][x]["grpn"]}')
                plt.axis('off')
                plt.show()

    return data

In [268]:
def arfread1(PATH, **kwargs):
    # defaults
    PLOT = kwargs.get('PLOT', False)
    RP = kwargs.get('RP', False)
    
    isRZ = not RP
    
    data = {'RecHead': {}, 'groups': []}

    # open file
    with open(PATH, 'rb') as fid:
        # open RecHead data
        data['RecHead']['ftype'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['ngrps'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['nrecs'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['grpseek'] = struct.unpack('200i', fid.read(4*200))
        data['RecHead']['recseek'] = struct.unpack('2000i', fid.read(4*2000))
        data['RecHead']['file_ptr'] = struct.unpack('i', fid.read(4))[0]

        data['groups'] = []
        bFirstPass = True
        for x in range(data['RecHead']['ngrps']):
            # jump to the group location in the file
            fid.seek(data['RecHead']['grpseek'][x], 0)

            # open the group
            data['groups'].append({
                'grpn': struct.unpack('h', fid.read(2))[0],
                'frecn': struct.unpack('h', fid.read(2))[0],
                'nrecs': struct.unpack('h', fid.read(2))[0],
                'ID': get_str(fid.read(16)),
                'ref1': get_str(fid.read(16)),
                'ref2': get_str(fid.read(16)),
                'memo': get_str(fid.read(50)),
            })

            # read temporary timestamp
            if bFirstPass:
                if isRZ:
                    ttt = struct.unpack('q', fid.read(8))[0]
                    fid.seek(-8, 1)
                    data['fileType'] = 'BioSigRZ'
                else:
                    ttt = struct.unpack('I', fid.read(4))[0]
                    fid.seek(-4, 1)
                    data['fileType'] = 'BioSigRP'
                data['fileTime'] = datetime.datetime.utcfromtimestamp(ttt/86400 + datetime.datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S')
                bFirstPass = False

            if isRZ:
                grp_t_format = 'q'
                beg_t_format = 'q'
                end_t_format = 'q'
                read_size = 8
            else:
                grp_t_format = 'I'
                beg_t_format = 'I'
                end_t_format = 'I'
                read_size = 4

            data['groups'][x]['beg_t'] = struct.unpack(beg_t_format, fid.read(read_size))[0]
            data['groups'][x]['end_t'] = struct.unpack(end_t_format, fid.read(read_size))[0]

            data['groups'][x].update({
                'sgfname1': get_str(fid.read(100)),
                'sgfname2': get_str(fid.read(100)),
                'VarName1': get_str(fid.read(15)),
                'VarName2': get_str(fid.read(15)),
                'VarName3': get_str(fid.read(15)),
                'VarName4': get_str(fid.read(15)),
                'VarName5': get_str(fid.read(15)),
                'VarName6': get_str(fid.read(15)),
                'VarName7': get_str(fid.read(15)),
                'VarName8': get_str(fid.read(15)),
                'VarName9': get_str(fid.read(15)),
                'VarName10': get_str(fid.read(15)),
                'VarUnit1': get_str(fid.read(5)),
                'VarUnit2': get_str(fid.read(5)),
                'VarUnit3': get_str(fid.read(5)),
                'VarUnit4': get_str(fid.read(5)),
                'VarUnit5': get_str(fid.read(5)),
                'VarUnit6': get_str(fid.read(5)),
                'VarUnit7': get_str(fid.read(5)),
                'VarUnit8': get_str(fid.read(5)),
                'VarUnit9': get_str(fid.read(5)),
                'VarUnit10': get_str(fid.read(5)),
                'SampPer_us': struct.unpack('f', fid.read(4))[0],
                'cc_t': struct.unpack('i', fid.read(4))[0],
                'version': struct.unpack('h', fid.read(2))[0],
                'postproc': struct.unpack('i', fid.read(4))[0],
                'dump': get_str(fid.read(92)),
                'recs': [],
            })

            for i in range(data['groups'][x]['nrecs']):
                record_data = {
                        'recn': struct.unpack('h', fid.read(2))[0],
                        'grpid': struct.unpack('h', fid.read(2))[0],
                        'grp_t': struct.unpack(grp_t_format, fid.read(read_size))[0],
                        #'grp_d': datetime.utcfromtimestamp(data['groups'][x]['recs'][i]['grp_t']/86400 + datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S'),
                        'newgrp': struct.unpack('h', fid.read(2))[0],
                        'sgi': struct.unpack('h', fid.read(2))[0],
                        'chan': struct.unpack('B', fid.read(1))[0],
                        'rtype': get_str(fid.read(1)),
                        'npts': struct.unpack('H' if isRZ else 'h', fid.read(2))[0],
                        'osdel': struct.unpack('f', fid.read(4))[0],
                        'dur_ms': struct.unpack('f', fid.read(4))[0],
                        'SampPer_us': struct.unpack('f', fid.read(4))[0],
                        'artthresh': struct.unpack('f', fid.read(4))[0],
                        'gain': struct.unpack('f', fid.read(4))[0],
                        'accouple': struct.unpack('h', fid.read(2))[0],
                        'navgs': struct.unpack('h', fid.read(2))[0],
                        'narts': struct.unpack('h', fid.read(2))[0],
                        'beg_t': struct.unpack(beg_t_format, fid.read(read_size))[0],
                        'end_t': struct.unpack(end_t_format, fid.read(read_size))[0],
                        'Var1': struct.unpack('f', fid.read(4))[0],
                        'Var2': struct.unpack('f', fid.read(4))[0],
                        'Var3': struct.unpack('f', fid.read(4))[0],
                        'Var4': struct.unpack('f', fid.read(4))[0],
                        'Var5': struct.unpack('f', fid.read(4))[0],
                        'Var6': struct.unpack('f', fid.read(4))[0],
                        'Var7': struct.unpack('f', fid.read(4))[0],
                        'Var8': struct.unpack('f', fid.read(4))[0],
                        'Var9': struct.unpack('f', fid.read(4))[0],
                        'Var10': struct.unpack('f', fid.read(4))[0],
                        'data': [] #list(struct.unpack(f'{data["groups"][x]["recs"][i]["npts"]}f', fid.read(4*data['groups'][x]['recs'][i]['npts'])))
                    }
                
                # skip all 10 cursors placeholders
                fid.seek(36*10, 1)
                record_data['data'] = list(struct.unpack(f'{record_data["npts"]}f', fid.read(4*record_data['npts'])))

                record_data['grp_d'] = datetime.datetime.utcfromtimestamp(record_data['grp_t'] / 86400 + datetime.datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S')

                data['groups'][x]['recs'].append(record_data)

            if PLOT:
                import matplotlib.pyplot as plt

                # determine reasonable spacing between plots
                d = [x['data'] for x in data['groups'][x]['recs']]
                plot_offset = max(max(map(abs, [item for sublist in d for item in sublist]))) * 1.2

                plt.figure()

                for i in range(data['groups'][x]['nrecs']):
                    plt.plot([item - plot_offset * i for item in data['groups'][x]['recs'][i]['data']])
                    plt.hold(True)

                plt.title(f'Group {data["groups"][x]["grpn"]}')
                plt.axis('off')
                plt.show()

    return data

In [ ]:
import numpy as np
import datetime
import matplotlib.pyplot as plt

def arfread2(PATH, **kwargs):
    PLOT = kwargs.get('PLOT', False)
    RP = kwargs.get('RP', False)

    isRZ = not RP

    data = {'RecHead': {}, 'groups': []}

    with open(PATH, 'rb') as fid:
        data['RecHead']['ftype'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
        data['RecHead']['ngrps'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
        data['RecHead']['nrecs'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
        data['RecHead']['grpseek'] = np.fromfile(fid, dtype=np.int32, count=200)
        data['RecHead']['recseek'] = np.fromfile(fid, dtype=np.int32, count=2000)
        data['RecHead']['file_ptr'] = np.fromfile(fid, dtype=np.int32, count=1)[0]

        data['groups'] = []
        bFirstPass = True

        for x in range(data['RecHead']['ngrps']):
            group = {}

            group['grpn'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
            group['frecn'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
            group['nrecs'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
            group['ID'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=16).astype('str'))
            group['ref1'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=16).astype('str'))
            group['ref2'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=16).astype('str'))
            group['memo'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=50).astype('str'))

            if bFirstPass:
                if isRZ:
                    ttt = np.fromfile(fid, dtype=np.int64, count=1)[0]
                    fid.seek(-8, 1)
                    data['fileTime'] = datetime.datetime.utcfromtimestamp(ttt / 86400 + datetime.datetime(1970, 1, 1).timestamp())
                    data['fileType'] = 'BioSigRZ'
                else:
                    ttt = np.fromfile(fid, dtype=np.uint32, count=1)[0]
                    fid.seek(-4, 1)
                    data['fileTime'] = datetime.datetime.utcfromtimestamp(ttt / 86400 + datetime.datetime(1970, 1, 1).timestamp())
                    data['fileType'] = 'BioSigRP'
                bFirstPass = False

            if isRZ:
                group['beg_t'] = np.fromfile(fid, dtype=np.int64, count=1)[0]
                group['end_t'] = np.fromfile(fid, dtype=np.int64, count=1)[0]
            else:
                group['beg_t'] = np.fromfile(fid, dtype=np.int32, count=1)[0]
                group['end_t'] = np.fromfile(fid, dtype=np.int32, count=1)[0]

            group['sgfname1'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=100).astype('str'))
            group['sgfname2'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=100).astype('str'))

            group['VarName1'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=15).astype('str'))
            group['VarName2'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=15).astype('str'))
            group['VarName3'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=15).astype('str'))
            group['VarName4'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=15).astype('str'))
            group['VarName5'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=15).astype('str'))
            group['VarName6'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=15).astype('str'))
            group['VarName7'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=15).astype('str'))
            group['VarName8'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=15).astype('str'))
            group['VarName9'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=15).astype('str'))
            group['VarName10'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=15).astype('str'))

            group['VarUnit1'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=5).astype('str'))
            group['VarUnit2'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=5).astype('str'))
            group['VarUnit3'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=5).astype('str'))
            group['VarUnit4'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=5).astype('str'))
            group['VarUnit5'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=5).astype('str'))
            group['VarUnit6'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=5).astype('str'))
            group['VarUnit7'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=5).astype('str'))
            group['VarUnit8'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=5).astype('str'))
            group['VarUnit9'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=5).astype('str'))
            group['VarUnit10'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=5).astype('str'))

            group['SampPer_us'] = np.fromfile(fid, dtype=np.float32, count=1)[0]

            group['cc_t'] = np.fromfile(fid, dtype=np.int32, count=1)[0]
            group['version'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
            group['postproc'] = np.fromfile(fid, dtype=np.int32, count=1)[0]
            group['dump'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=92).astype('str'))

            group['recs'] = []
            for i in range(group['nrecs']):
                rec = {}

                rec['recn'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
                rec['grpid'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
                if isRZ:
                    rec['grp_t'] = np.fromfile(fid, dtype=np.int64, count=1)[0]
                else:
                    rec['grp_t'] = np.fromfile(fid, dtype=np.int32, count=1)[0]
                rec['newgrp'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
                rec['sgi'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
                rec['chan'] = np.fromfile(fid, dtype=np.uint8, count=1)[0]
                rec['rtype'] = ''.join(np.fromfile(fid, dtype=np.uint8, count=1).astype('str'))

                rec['npts'] = np.fromfile(fid, dtype=np.uint16, count=1)[0] if isRZ else np.fromfile(fid, dtype=np.int16, count=1)[0]
                rec['osdel'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['dur_ms'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['SampPer_us'] = np.fromfile(fid, dtype=np.float32, count=1)[0]

                rec['artthresh'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['gain'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['accouple'] = np.fromfile(fid, dtype=np.int16, count=1)[0]

                rec['navgs'] = np.fromfile(fid, dtype=np.int16, count=1)[0]
                rec['narts'] = np.fromfile(fid, dtype=np.int16, count=1)[0]

                if isRZ:
                    rec['beg_t'] = np.fromfile(fid, dtype=np.int64, count=1)[0]
                    rec['end_t'] = np.fromfile(fid, dtype=np.int64, count=1)[0]
                else:
                    rec['beg_t'] = np.fromfile(fid, dtype=np.int32, count=1)[0]
                    rec['end_t'] = np.fromfile(fid, dtype=np.int32, count=1)[0]

                rec['Var1'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['Var2'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['Var3'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['Var4'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['Var5'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['Var6'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['Var7'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['Var8'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['Var9'] = np.fromfile(fid, dtype=np.float32, count=1)[0]
                rec['Var10'] = np.fromfile(fid, dtype=np.float32, count=1)[0]

                fid.seek(36 * 10, 1)

                rec['data'] = np.fromfile(fid, dtype=np.float32, count=group['npts'])

                group['recs'].append(rec)

            data['groups'].append(group)

            if PLOT:
                plt.figure()

                plot_offset = max([max(abs(rec['data'])) for rec in group['recs']]) * 1.2

                for i, rec in enumerate(group['recs']):
                    plt.plot(rec['data'] - plot_offset * (i + 1))
                    plt.hold(True)

                plt.title('Group {}'.format(group['grpn']))
                plt.axis('off')

    return data


# ABRA base code for eventual wave I isolation

## Preliminary Model Process

1. Normalize/scale the graph data to use threshold, amplitude as the predictors
2. Simplify images to grayscale and resize to a like image
3. Build a simple autoencoder to predict images
4. Fit and train

AFTER
- Fine tune inputs
- Have Wave I morphology as an input
- Consider PCA or some dimensionality reduction
- Create script to average wave data by level for external lab data


Note to self: figure out how to just predict image features like counts, is that a more efficient approach?

# 0. Collect ABR data, group by frequency/dB

## Read ABR

## RZ LOGIC (only paired data we have so far)

In [ ]:
def arfread1(PATH, **kwargs):
    # defaults
    PLOT = kwargs.get('PLOT', False)
    RP = kwargs.get('RP', False)
    
    isRZ = not RP
    
    data = {'RecHead': {}, 'groups': []}

    # open file
    with open(PATH, 'rb') as fid:
        # open RecHead data
        data['RecHead']['ftype'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['ngrps'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['nrecs'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['grpseek'] = struct.unpack('200i', fid.read(4*200))
        data['RecHead']['recseek'] = struct.unpack('2000i', fid.read(4*2000))
        data['RecHead']['file_ptr'] = struct.unpack('i', fid.read(4))[0]

        data['groups'] = []
        bFirstPass = True
        for x in range(data['RecHead']['ngrps']):
            # jump to the group location in the file
            fid.seek(data['RecHead']['grpseek'][x], 0)

            # open the group
            data['groups'].append({
                'grpn': struct.unpack('h', fid.read(2))[0],
                'frecn': struct.unpack('h', fid.read(2))[0],
                'nrecs': struct.unpack('h', fid.read(2))[0],
                'ID': get_str(fid.read(16)),
                'ref1': get_str(fid.read(16)),
                'ref2': get_str(fid.read(16)),
                'memo': get_str(fid.read(50)),
            })

            # read temporary timestamp
            if bFirstPass:
                if isRZ:
                    ttt = struct.unpack('q', fid.read(8))[0]
                    fid.seek(-8, 1)
                    data['fileType'] = 'BioSigRZ'
                else:
                    ttt = struct.unpack('I', fid.read(4))[0]
                    fid.seek(-4, 1)
                    data['fileType'] = 'BioSigRP'
                data['fileTime'] = datetime.datetime.utcfromtimestamp(ttt/86400 + datetime.datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S')
                bFirstPass = False

            if isRZ:
                grp_t_format = 'q'
                beg_t_format = 'q'
                end_t_format = 'q'
                read_size = 8
            else:
                grp_t_format = 'I'
                beg_t_format = 'I'
                end_t_format = 'I'
                read_size = 4

            data['groups'][x]['beg_t'] = struct.unpack(beg_t_format, fid.read(read_size))[0]
            data['groups'][x]['end_t'] = struct.unpack(end_t_format, fid.read(read_size))[0]

            data['groups'][x].update({
                'sgfname1': get_str(fid.read(100)),
                'sgfname2': get_str(fid.read(100)),
                'VarName1': get_str(fid.read(15)),
                'VarName2': get_str(fid.read(15)),
                'VarName3': get_str(fid.read(15)),
                'VarName4': get_str(fid.read(15)),
                'VarName5': get_str(fid.read(15)),
                'VarName6': get_str(fid.read(15)),
                'VarName7': get_str(fid.read(15)),
                'VarName8': get_str(fid.read(15)),
                'VarName9': get_str(fid.read(15)),
                'VarName10': get_str(fid.read(15)),
                'VarUnit1': get_str(fid.read(5)),
                'VarUnit2': get_str(fid.read(5)),
                'VarUnit3': get_str(fid.read(5)),
                'VarUnit4': get_str(fid.read(5)),
                'VarUnit5': get_str(fid.read(5)),
                'VarUnit6': get_str(fid.read(5)),
                'VarUnit7': get_str(fid.read(5)),
                'VarUnit8': get_str(fid.read(5)),
                'VarUnit9': get_str(fid.read(5)),
                'VarUnit10': get_str(fid.read(5)),
                'SampPer_us': struct.unpack('f', fid.read(4))[0],
                'cc_t': struct.unpack('i', fid.read(4))[0],
                'version': struct.unpack('h', fid.read(2))[0],
                'postproc': struct.unpack('i', fid.read(4))[0],
                'dump': get_str(fid.read(92)),
                'recs': [],
            })

            for i in range(data['groups'][x]['nrecs']):
                record_data = {
                        'recn': struct.unpack('h', fid.read(2))[0],
                        'grpid': struct.unpack('h', fid.read(2))[0],
                        'grp_t': struct.unpack(grp_t_format, fid.read(read_size))[0],
                        #'grp_d': datetime.utcfromtimestamp(data['groups'][x]['recs'][i]['grp_t']/86400 + datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S'),
                        'newgrp': struct.unpack('h', fid.read(2))[0],
                        'sgi': struct.unpack('h', fid.read(2))[0],
                        'chan': struct.unpack('B', fid.read(1))[0],
                        'rtype': get_str(fid.read(1)),
                        'npts': struct.unpack('H' if isRZ else 'h', fid.read(2))[0],
                        'osdel': struct.unpack('f', fid.read(4))[0],
                        'dur_ms': struct.unpack('f', fid.read(4))[0],
                        'SampPer_us': struct.unpack('f', fid.read(4))[0],
                        'artthresh': struct.unpack('f', fid.read(4))[0],
                        'gain': struct.unpack('f', fid.read(4))[0],
                        'accouple': struct.unpack('h', fid.read(2))[0],
                        'navgs': struct.unpack('h', fid.read(2))[0],
                        'narts': struct.unpack('h', fid.read(2))[0],
                        'beg_t': struct.unpack(beg_t_format, fid.read(read_size))[0],
                        'end_t': struct.unpack(end_t_format, fid.read(read_size))[0],
                        'Var1': struct.unpack('f', fid.read(4))[0],
                        'Var2': struct.unpack('f', fid.read(4))[0],
                        'Var3': struct.unpack('f', fid.read(4))[0],
                        'Var4': struct.unpack('f', fid.read(4))[0],
                        'Var5': struct.unpack('f', fid.read(4))[0],
                        'Var6': struct.unpack('f', fid.read(4))[0],
                        'Var7': struct.unpack('f', fid.read(4))[0],
                        'Var8': struct.unpack('f', fid.read(4))[0],
                        'Var9': struct.unpack('f', fid.read(4))[0],
                        'Var10': struct.unpack('f', fid.read(4))[0],
                        'data': [] #list(struct.unpack(f'{data["groups"][x]["recs"][i]["npts"]}f', fid.read(4*data['groups'][x]['recs'][i]['npts'])))
                    }
                
                # skip all 10 cursors placeholders
                fid.seek(36*10, 1)
                record_data['data'] = list(struct.unpack(f'{record_data["npts"]}f', fid.read(4*record_data['npts'])))

                record_data['grp_d'] = datetime.datetime.utcfromtimestamp(record_data['grp_t'] / 86400 + datetime.datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S')

                data['groups'][x]['recs'].append(record_data)

            if PLOT:
                import matplotlib.pyplot as plt

                # determine reasonable spacing between plots
                d = [x['data'] for x in data['groups'][x]['recs']]
                plot_offset = max(max(map(abs, [item for sublist in d for item in sublist]))) * 1.2

                plt.figure()

                for i in range(data['groups'][x]['nrecs']):
                    plt.plot([item - plot_offset * i for item in data['groups'][x]['recs'][i]['data']])
                    plt.hold(True)

                plt.title(f'Group {data["groups"][x]["grpn"]}')
                plt.axis('off')
                plt.show()

    rows = [] 
    freqs = []
    dbs = []
    for group in data['groups']: # there are multiple groups under each file, varies per file
        for rec in group['recs']:
            # freq = rec['Var1']
            freq = '?'
            freqs.append(freq)

            db = rec['Var1']
            dbs.append(db)
            
            wave_cols = list(enumerate(rec['data']))
            wave_data = {f'{i}':v*1e6 for i, v in wave_cols} 

            row = {'Freq(Hz)': freq, 'Level(dB)': db, **wave_data}
            rows.append(row)
    final = pd.DataFrame(rows)
    return final

In [ ]:
arfread1('/Users/leahashebir/Downloads/Manor_Practicum/manor_lauren/abr_data/639F_click.arf')

,Freq (Hz),Level (dB),0,1,2,3,4,5,6,7,...,234,235,236,237,238,239,240,241,242,243
0,?,90.0,0.005186,0.005272,0.005933,0.006823,0.007570,0.008109,0.008606,0.009195,...,-0.000626,-0.000523,-0.000344,-0.000174,0.000084,0.000581,0.001499,0.002598,0.003618,0.004559
1,?,85.0,-0.003063,-0.003353,-0.003532,-0.003290,-0.003001,-0.003027,-0.003239,-0.003247,...,0.007389,0.006607,0.005638,0.004487,0.003040,0.001399,-0.000035,-0.000816,-0.000867,-0.001015


In [ ]:
abr_path = "/Users/leahashebir/Downloads/Manor_Practicum/manor_lauren/abr_data" # Temp address
abr_directory = os.listdir(abr_path)

rows = [] 
freqs = []
dbs = []
for file in abr_directory:
    path = os.path.abspath(os.path.join(abr_path,file))
    data = arfread1(path) #Believe it is RZ since RP doesn't work

    abrs = []
    for group in data['groups']: # there are multiple groups under each file, varies per file
        for rec in group['recs']:
            # freq = rec['Var1']
            freq = '?'
            freqs.append(freq)

            db = rec['Var1']
            dbs.append(db)
            
            wave_cols = list(enumerate(rec['data']))
            wave_data = {f'{i}':v*1e6 for i, v in wave_cols} 

            row = {'Freq (Hz)': freq, 'Level (dB)': db, **wave_data}
            rows.append(row)

final = pd.DataFrame(rows)
final

,Freq (Hz),Level (dB),0,1,2,3,4,5,6,7,...,234,235,236,237,238,239,240,241,242,243
0,?,90.0,0.006391,0.007161,0.007582,0.007445,0.006667,0.005551,0.004286,0.003200,...,0.000786,0.001755,0.002662,0.003138,0.003261,0.003596,0.004502,0.005792,0.007048,0.008007
1,?,85.0,0.004196,0.003979,0.003646,0.003183,0.002656,0.002060,0.001504,0.001221,...,-0.021559,-0.021497,-0.021739,-0.021934,-0.021847,-0.021407,-0.020667,-0.019630,-0.018367,-0.017303
2,?,80.0,-0.013517,-0.014442,-0.015600,-0.016704,-0.017431,-0.017591,-0.017088,-0.016014,...,-0.002121,-0.001654,-0.001294,-0.001097,-0.001104,-0.001249,-0.001319,-0.001135,-0.000633,0.000178
3,?,75.0,0.002894,0.002781,0.002950,0.003281,0.003671,0.004070,0.004468,0.004706,...,0.008352,0.008418,0.008498,0.008760,0.009010,0.008944,0.008675,0.008346,0.007894,0.007288
4,?,70.0,0.010361,0.012123,0.013713,0.015028,0.016261,0.017410,0.018192,0.018626,...,-0.002404,-0.001619,-0.000490,0.000746,0.001818,0.002376,0.002433,0.002580,0.003308,0.004394
5,?,90.0,0.005186,0.005272,0.005933,0.006823,0.007570,0.008109,0.008606,0.009195,...,-0.000626,-0.000523,-0.000344,-0.000174,0.000084,0.000581,0.001499,0.002598,0.003618,0.004559
6,?,85.0,-0.003063,-0.003353,-0.003532,-0.003290,-0.003001,-0.003027,-0.003239,-0.003247,...,0.007389,0.006607,0.005638,0.004487,0.003040,0.001399,-0.000035,-0.000816,-0.000867,-0.001015
7,?,90.0,0.010354,0.011036,0.011346,0.011510,0.011681,0.011764,0.011733,0.011689,...,-0.039049,-0.036867,-0.034337,-0.031873,-0.029744,-0.028037,-0.026541,-0.024916,-0.023024,-0.020857
8,?,85.0,0.006431,0.007364,0.007726,0.007754,0.007683,0.007766,0.008296,0.009206,...,-0.048247,-0.046323,-0.044214,-0.042286,-0.040737,-0.039433,-0.037834,-0.035614,-0.032739,-0.029490
9,?,80.0,-0.000886,-0.001449,-0.001849,-0.002149,-0.002311,-0.002519,-0.003018,-0.003783,...,-0.039156,-0.037802,-0.036243,-0.034529,-0.032880,-0.031559,-0.030565,-0.029605,-0.028627,-0.027906


In [ ]:
arfread1('/Users/leahashebir/Downloads/Manor_Practicum/manor_lauren/abr_data/639F_click.arf')

{'RecHead': {'ftype': 1,
  'ngrps': 1,
  'nrecs': 2,
  'grpseek': (8811,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   

## RP LOGIC

In [ ]:
arfread2()

## Average across the groups by dB [should already be averaged across for Manor data though]
Separate process from the typical script for non-Manor data

In [ ]:
scaled = {}
collect_scaled = []
all_scaled = {}
for data in all_abr:
    print(data)

[{'0': 0.006391209783629392, '1': 0.007160744441847555, '2': 0.007581756555907759, '3': 0.00744545181063927, '4': 0.0066668404130609815, '5': 0.0055508757590416735, '6': 0.004286448085366601, '7': 0.0032002327454705437, '8': 0.002644932495243779, '9': 0.0025377451251529237, '10': 0.0025431532435504778, '11': 0.00254419751932744, '12': 0.002511751029388165, '13': 0.002438597324072589, '14': 0.0022521371434436332, '15': 0.001881772737633014, '16': 0.0011790246556842021, '17': -0.0001212706463027402, '18': -0.0018173602622795215, '19': -0.003206075849249146, '20': -0.004026635025411451, '21': -0.0046443955348252075, '22': -0.0055318816194471765, '23': -0.006668358309980249, '24': -0.00768065078204927, '25': -0.008349606339663751, '26': -0.00850039683086834, '27': -0.008313588040209652, '28': -0.008014376717824234, '29': -0.007482568342709328, '30': -0.0066794756392596355, '31': -0.005875434361257703, '32': -0.00548279244227956, '33': -0.005791469526172932, '34': -0.006584131906350876, '35

## Group by frequency/dB
(predict by freq/db intersection and then generalize? Or can we generalize, idkkkkk. Q for Cody)

# 1a Calculate Wave I threshold and amplitude (includes normalization)
No need to isolate yet because testing using metrics first before the wave itself

In [ ]:
from tensorflow.keras.models import load_model
from scipy.interpolate import CubicSpline
from scipy.signal import find_peaks

def interpolate_and_smooth(final, target_length=244): # To implement after moving beyond Manor data since we already have 244 time points
    if len(final) > target_length:
        new_points = np.linspace(0, len(final), target_length + 2)
        interpolated_values = np.interp(new_points, np.arange(len(final)), final)
        final = np.array(interpolated_values[:target_length], dtype=float)
    elif len(final) < target_length:
        original_indices = np.arange(len(final))
        target_indices = np.linspace(0, len(final) - 1, target_length)
        cs = CubicSpline(original_indices, final)
        final = cs(target_indices)
    return final

def calculate_hearing_threshold(df, freq, baseline_level=100, multiply_y_factor=1):
    db_column = 'Level(dB)' if level else 'PostAtten(dB)'

    thresholding_model = load_model('./models/abr_cnn_aug_norm_opt.keras')
    thresholding_model.steps_per_execution = 1
    
    # Filter DataFrame to include only data for the specified frequency
    df_filtered = df[df['Freq(Hz)'] == freq]

    # Get unique dB levels for the filtered DataFrame
    db_levels = sorted(df_filtered[db_column].unique()) if db_column == 'Level(dB)' else sorted(df_filtered[db_column].unique(), reverse=True)
    
    lowest_db = None
    waves = []
    previous_prediction = None  # Initialize previous prediction to track consecutive `1`s

    for db in db_levels:
        khz = df_filtered[df_filtered[db_column] == np.abs(db)]
        if not khz.empty:
            index = khz.index.values[-1]
            final = df_filtered.loc[index, '0':].dropna()
            final = pd.to_numeric(final, errors='coerce')
            final = np.array(final, dtype=np.float64)
            final = interpolate_and_smooth(final)

            if units == 'Nanovolts':
                final /= 1000

            waves.append(final)
    
    waves = np.array(waves)
    flattened_data = waves.flatten().reshape(-1, 1)
    scaler = StandardScaler()
    scaled_flattened_data = scaler.fit_transform(flattened_data).reshape(waves.shape)
    waves = np.expand_dims(scaled_flattened_data, axis=2)
    
    # Perform prediction
    prediction = thresholding_model.predict(waves)
    y_pred = (prediction > 0.5).astype(int).flatten()

    calibration_levels = {} # Used only for post-atten. Focus on this after working with Lauren data
    for p, d in zip(y_pred, db_levels):
        if p == 0:
            if db_column == 'PostAtten(dB)':
                calibration_key = (df.name, freq)  # Assuming df has an attribute 'name' set to the file name
                if calibration_key in calibration_levels:
                    calibration_level = calibration_levels[calibration_key]
                else:
                    calibration_level = 0
                lowest_db = baseline_level - (d + calibration_level)
            else:
                lowest_db = d
            previous_prediction = p  # Update previous prediction
        else:
            if previous_prediction == 1:
                break  # Break if two consecutive `1`s are encountered
            if db_column == 'PostAtten(dB)':
                calibration_key = (df.name, freq)
                if calibration_key in calibration_levels:
                    calibration_level = calibration_levels[calibration_key]
                else:
                    calibration_level = 0
                lowest_db = baseline_level - (d + calibration_level)
            else:
                lowest_db = d
            previous_prediction = p  # Update previous prediction
    
    return lowest_db

class CNN(nn.Module):
    def __init__(self, dropout_prob=0.1):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 61, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(-1, 32 * 61)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

peak_finding_model = CNN()

def peak_finding(wave):
    # Prepare waveform
    waveform=interpolate_and_smooth(wave)
    waveform_torch = torch.tensor(waveform, dtype=torch.float32).unsqueeze(0)
    
    # Get prediction from model
    outputs = peak_finding_model(waveform_torch)
    prediction = int(round(outputs.detach().numpy()[0][0], 0))

    # Apply Gaussian smoothing
    smoothed_waveform = gaussian_filter1d(waveform, sigma=1)

    # Find peaks and troughs
    n = 18
    t = 14
    start_point = prediction - 9
    smoothed_peaks, _ = find_peaks(smoothed_waveform[start_point:], distance=n)
    smoothed_troughs, _ = find_peaks(-smoothed_waveform, distance=t)
    sorted_indices = np.argsort(smoothed_waveform[smoothed_peaks+start_point])
    highest_smoothed_peaks = np.sort(smoothed_peaks[sorted_indices[-5:]] + start_point)
    relevant_troughs = np.array([])
    for p in range(len(highest_smoothed_peaks)):
        c = 0
        for t in smoothed_troughs:
            if t > highest_smoothed_peaks[p]:
                if p != 4:
                    try:
                        if t < highest_smoothed_peaks[p+1]:
                            relevant_troughs = np.append(relevant_troughs, int(t))
                            break
                    except IndexError:
                        pass
                else:
                    relevant_troughs = np.append(relevant_troughs, int(t))
                    break
    relevant_troughs = relevant_troughs.astype('i')
    return highest_smoothed_peaks, relevant_troughs



In [ ]:
abr_path = "/Users/leahashebir/Downloads/Manor_Practicum/manor_lauren/abr_data" #clean up
abr_directory = os.listdir(abr_path)
dfs = []
for file in abr_directory:
    path = os.path.abspath(os.path.join(abr_path,file))
    data = arfread1(path)
    data_df = pd.DataFrame(data)
    dfs.append(data_df)

final_df = pd.concat(dfs, ignore_index = True)

In [ ]:
def calculate_hearing_threshold_1(df, freq, baseline_level=100, multiply_y_factor=1, level = True, units = "Microvolts"):
    db_column = 'Level(dB)' if level else 'PostAtten(dB)'

    thresholding_model = load_model('./models/abr_cnn_aug_norm_opt.keras')
    thresholding_model.steps_per_execution = 1
    
    # Filter DataFrame to include only data for the specified frequency
    df_filtered = df[df['Freq(Hz)'] == freq]

    # Get unique dB levels for the filtered DataFrame
    db_levels = sorted(df_filtered[db_column].unique()) if db_column == 'Level(dB)' else sorted(df_filtered[db_column].unique(), reverse=True)
    
    lowest_db = None
    waves = []
    previous_prediction = None  # Initialize previous prediction to track consecutive `1`s

    for db in db_levels:
        khz = df_filtered[df_filtered[db_column] == np.abs(db)]
        if not khz.empty:
            index = khz.index.values[-1]
            final = df_filtered.loc[index, '0':].dropna()
            final = pd.to_numeric(final, errors='coerce')
            final = np.array(final, dtype=np.float64)
            final = interpolate_and_smooth(final)

            if units == 'Nanovolts':
                final /= 1000

            waves.append(final)
    
    waves = np.array(waves)
    flattened_data = waves.flatten().reshape(-1, 1)
    scaler = StandardScaler()
    scaled_flattened_data = scaler.fit_transform(flattened_data).reshape(waves.shape)
    waves = np.expand_dims(scaled_flattened_data, axis=2)
    
    # Perform prediction
    prediction = thresholding_model.predict(waves)
    y_pred = (prediction > 0.5).astype(int).flatten()

    calibration_levels = {} # Used only for post-atten. Focus on this after working with Lauren data
    for p, d in zip(y_pred, db_levels):
        if p == 0:
            if db_column == 'PostAtten(dB)':
                calibration_key = (df.name, freq)  # Assuming df has an attribute 'name' set to the file name
                if calibration_key in calibration_levels:
                    calibration_level = calibration_levels[calibration_key]
                else:
                    calibration_level = 0
                lowest_db = baseline_level - (d + calibration_level)
            else:
                lowest_db = d
            previous_prediction = p  # Update previous prediction
        else:
            if previous_prediction == 1:
                break  # Break if two consecutive `1`s are encountered
            if db_column == 'PostAtten(dB)':
                calibration_key = (df.name, freq)
                if calibration_key in calibration_levels:
                    calibration_level = calibration_levels[calibration_key]
                else:
                    calibration_level = 0
                lowest_db = baseline_level - (d + calibration_level)
            else:
                lowest_db = d
            previous_prediction = p  # Update previous prediction
    
    return lowest_db

In [ ]:
calculate_hearing_threshold_1(df = final_df, freq = '?', level= True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step


np.float64(55.0)

In [ ]:
khz = final_df[(final_df['Freq(Hz)'] == '?') & (final_df['Level(dB)'] == 55)]
if not khz.empty:
    index = khz.index.values[0]
    final = final_df.loc[index, '0':].dropna()
    final = pd.to_numeric(final, errors='coerce').dropna()
    wave = interpolate_and_smooth(final)
    highest_smoothed_peaks, relevant_troughs = peak_finding(wave)

highest_smoothed_peaks, relevant_troughs

/var/folders/zf/2fdw6l214h14v9prl_rrp3180000gp/T/ipykernel_90977/142231517.py:112: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  waveform_torch = torch.tensor(waveform, dtype=torch.float32).unsqueeze(0)


(array([], dtype=int64), array([], dtype=int32))

In [278]:
def arfread3(PATH, **kwargs):
    # defaults
    PLOT = kwargs.get('PLOT', False)
    RP = kwargs.get('RP', False)
    
    isRZ = not RP
    
    data = {'RecHead': {}, 'groups': []}

    # open file
    with open(PATH, 'rb') as fid:
        # open RecHead data
        data['RecHead']['ftype'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['ngrps'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['nrecs'] = struct.unpack('h', fid.read(2))[0]
        data['RecHead']['grpseek'] = struct.unpack('200i', fid.read(4*200))
        data['RecHead']['recseek'] = struct.unpack('2000i', fid.read(4*2000))
        data['RecHead']['file_ptr'] = struct.unpack('i', fid.read(4))[0]

        data['groups'] = []
        bFirstPass = True
        for x in range(data['RecHead']['ngrps']):
            # jump to the group location in the file
            fid.seek(data['RecHead']['grpseek'][x], 0)

            # open the group
            data['groups'].append({
                'grpn': struct.unpack('h', fid.read(2))[0],
                'frecn': struct.unpack('h', fid.read(2))[0],
                'nrecs': struct.unpack('h', fid.read(2))[0],
                'ID': get_str(fid.read(16)),
                'ref1': get_str(fid.read(16)),
                'ref2': get_str(fid.read(16)),
                'memo': get_str(fid.read(50)),
            })

            # read temporary timestamp
            if bFirstPass:
                if isRZ:
                    ttt = struct.unpack('q', fid.read(8))[0]
                    fid.seek(-8, 1)
                    data['fileType'] = 'BioSigRZ'
                else:
                    ttt = struct.unpack('I', fid.read(4))[0]
                    fid.seek(-4, 1)
                    data['fileType'] = 'BioSigRP'
                data['fileTime'] = datetime.datetime.utcfromtimestamp(ttt/86400 + datetime.datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S')
                bFirstPass = False

            if isRZ:
                grp_t_format = 'q'
                beg_t_format = 'q'
                end_t_format = 'q'
                read_size = 8
            else:
                grp_t_format = 'I'
                beg_t_format = 'I'
                end_t_format = 'I'
                read_size = 4

            data['groups'][x]['beg_t'] = struct.unpack(beg_t_format, fid.read(read_size))[0]
            data['groups'][x]['end_t'] = struct.unpack(end_t_format, fid.read(read_size))[0]

            data['groups'][x].update({
                'sgfname1': get_str(fid.read(100)),
                'sgfname2': get_str(fid.read(100)),
                'VarName1': get_str(fid.read(15)),
                'VarName2': get_str(fid.read(15)),
                'VarName3': get_str(fid.read(15)),
                'VarName4': get_str(fid.read(15)),
                'VarName5': get_str(fid.read(15)),
                'VarName6': get_str(fid.read(15)),
                'VarName7': get_str(fid.read(15)),
                'VarName8': get_str(fid.read(15)),
                'VarName9': get_str(fid.read(15)),
                'VarName10': get_str(fid.read(15)),
                'VarUnit1': get_str(fid.read(5)),
                'VarUnit2': get_str(fid.read(5)),
                'VarUnit3': get_str(fid.read(5)),
                'VarUnit4': get_str(fid.read(5)),
                'VarUnit5': get_str(fid.read(5)),
                'VarUnit6': get_str(fid.read(5)),
                'VarUnit7': get_str(fid.read(5)),
                'VarUnit8': get_str(fid.read(5)),
                'VarUnit9': get_str(fid.read(5)),
                'VarUnit10': get_str(fid.read(5)),
                'SampPer_us': struct.unpack('f', fid.read(4))[0],
                'cc_t': struct.unpack('i', fid.read(4))[0],
                'version': struct.unpack('h', fid.read(2))[0],
                'postproc': struct.unpack('i', fid.read(4))[0],
                'dump': get_str(fid.read(92)),
                'recs': [],
            })

            for i in range(data['groups'][x]['nrecs']):
                record_data = {
                        'recn': struct.unpack('h', fid.read(2))[0],
                        'grpid': struct.unpack('h', fid.read(2))[0],
                        'grp_t': struct.unpack(grp_t_format, fid.read(read_size))[0],
                        #'grp_d': datetime.utcfromtimestamp(data['groups'][x]['recs'][i]['grp_t']/86400 + datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S'),
                        'newgrp': struct.unpack('h', fid.read(2))[0],
                        'sgi': struct.unpack('h', fid.read(2))[0],
                        'chan': struct.unpack('B', fid.read(1))[0],
                        'rtype': get_str(fid.read(1)),
                        'npts': struct.unpack('H' if isRZ else 'h', fid.read(2))[0],
                        'osdel': struct.unpack('f', fid.read(4))[0],
                        'dur_ms': struct.unpack('f', fid.read(4))[0],
                        'SampPer_us': struct.unpack('f', fid.read(4))[0],
                        'artthresh': struct.unpack('f', fid.read(4))[0],
                        'gain': struct.unpack('f', fid.read(4))[0],
                        'accouple': struct.unpack('h', fid.read(2))[0],
                        'navgs': struct.unpack('h', fid.read(2))[0],
                        'narts': struct.unpack('h', fid.read(2))[0],
                        'beg_t': struct.unpack(beg_t_format, fid.read(read_size))[0],
                        'end_t': struct.unpack(end_t_format, fid.read(read_size))[0],
                        'Var1': struct.unpack('f', fid.read(4))[0],
                        'Var2': struct.unpack('f', fid.read(4))[0],
                        'Var3': struct.unpack('f', fid.read(4))[0],
                        'Var4': struct.unpack('f', fid.read(4))[0],
                        'Var5': struct.unpack('f', fid.read(4))[0],
                        'Var6': struct.unpack('f', fid.read(4))[0],
                        'Var7': struct.unpack('f', fid.read(4))[0],
                        'Var8': struct.unpack('f', fid.read(4))[0],
                        'Var9': struct.unpack('f', fid.read(4))[0],
                        'Var10': struct.unpack('f', fid.read(4))[0],
                        'data': [] #list(struct.unpack(f'{data["groups"][x]["recs"][i]["npts"]}f', fid.read(4*data['groups'][x]['recs'][i]['npts'])))
                    }
                
                # skip all 10 cursors placeholders
                fid.seek(36*10, 1)
                record_data['data'] = list(struct.unpack(f'{record_data["npts"]}f', fid.read(4*record_data['npts'])))

                record_data['grp_d'] = datetime.datetime.utcfromtimestamp(record_data['grp_t'] / 86400 + datetime.datetime(1970, 1, 1).timestamp()).strftime('%Y-%m-%d %H:%M:%S')

                data['groups'][x]['recs'].append(record_data)

            if PLOT:
                import matplotlib.pyplot as plt

                # determine reasonable spacing between plots
                d = [x['data'] for x in data['groups'][x]['recs']]
                plot_offset = max(max(map(abs, [item for sublist in d for item in sublist]))) * 1.2

                plt.figure()

                for i in range(data['groups'][x]['nrecs']):
                    plt.plot([item - plot_offset * i for item in data['groups'][x]['recs'][i]['data']])
                    plt.hold(True)

                plt.title(f'Group {data["groups"][x]["grpn"]}')
                plt.axis('off')
                plt.show()

    rows = [] 
    freqs = []
    dbs = []
    for group in data['groups']: # there are multiple groups under each file, varies per file
        for rec in group['recs']:
            # freq = rec['Var1']
            freq = '?'
            freqs.append(freq)

            db = rec['Var1']
            dbs.append(db)
            
            wave_cols = list(enumerate(rec['data']))
            wave_data = {f'{i}':v*1e6 for i, v in wave_cols} 

            row = {'Freq(Hz)': freq, 'Level(dB)': db, **wave_data}
            rows.append(row)
    final = pd.DataFrame(rows)
    return final

def peaks_and_troughs(df, freq, db):
    khz = df[(df['Freq(Hz)'] == freq) & (df['Level(dB)'] == db)]
    if not khz.empty:
        index = khz.index.values[0]
        final = df.loc[index, '0':].dropna()
        final = pd.to_numeric(final, errors='coerce').dropna()
        wave = interpolate_and_smooth(final)
        highest_smoothed_peaks, relevant_troughs = peak_finding(wave)
        return highest_smoothed_peaks, relevant_troughs

# 1b Prep image and numeric data

Normalize images

In [275]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

image_directory = '/Users/leahashebir/Downloads/Manor_Practicum/manor_lauren/image_data'
data_bs = Path(image_directory)
splitfolders.ratio(data_bs, output=('/Users/leahashebir/Downloads/Manor_Practicum/modeling/new_image_train.test.validate'), seed=1234, ratio=(0.7, 0.15, 0.15), group_prefix=None)

train_path = "/Users/leahashebir/Downloads/Manor_Practicum/modeling/new_image_train.test.validate/train"
val_path = "/Users/leahashebir/Downloads/Manor_Practicum/modeling/new_image_train.test.validate/val"

img_height = 64 
img_width = 64
batch_size = 2  # How many images are in each batch for the training. Update to 20!!!
channels = 3 #1 for grayscale, 3 for RGB. Default data_gen makes it grayscale!!
input_shape = (batch_size, img_height, img_width, channels)

data_gen = ImageDataGenerator(rescale=1./255)
train_ds = data_gen.flow_from_directory(train_path, target_size=(img_height, img_width), class_mode='sparse', batch_size=batch_size, subset='training')
val_ds = data_gen.flow_from_directory(val_path, target_size=(img_height, img_width), class_mode='sparse', batch_size=batch_size, shuffle=False)

Found 25 images belonging to 7 classes.
Found 1 images belonging to 7 classes.


Concatenate all threshold/peak/trough data into an array for processing

In [283]:
abr_path = "/Users/leahashebir/Downloads/Manor_Practicum/manor_lauren/abr_data/Samp8 A1 baseline"
abr_directory = os.listdir(abr_path)
for file in abr_directory:
    print(f"Starting on file {file}")
    results = []
    path = os.path.abspath(os.path.join(abr_path,file))
    data = arfread3(path)
    data_df = pd.DataFrame(data)
    thresh = calculate_hearing_threshold_1(df = data_df, freq = '?', level= True)
    print(f'Calculated threshold={thresh}')
    highs, lows = peaks_and_troughs(df=data_df, db= 55, freq='?')
    # highs = peaks_and_troughs(data_df, db= 55, freq='?')
    # print(highs)
    print(f'Peak = {highs}, trough = {lows}')
    data = (file, thresh, highs, lows)
    results.append(data) # feed results in as the input array

results =  np.array(results)


Starting on file 1301_tone.arf
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Calculated threshold=32000.0


TypeError: cannot unpack non-iterable NoneType object

# 3. Build autoencoder

In [ ]:
from torch.optim import Adam

class MultiModalValueToImageFeaturePredict(nn.Module):
    def __init__(self, numerical_input_size=3, img_input_size=64):
        """
        Autoencoder to map specific input values to images.

        Args:
            input_size (int): Number of specific values to be used as input.
            img_size (int): Size of one side of the square image (e.g., for 28x28, img_size=28).
        """
        super(MultiModalValueToImageFeaturePredict, self).__init__()
        self.numerical_encoder = nn.Sequential(
            nn.Linear(numerical_input_size, 128), # 128 is a somewhat middle ground, can adjust
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.img_encoder = nn.Sequential(
            nn.Linear(img_input_size * img_input_size, 128),  # Flatten image into vector if it's input
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.img_size = img_input_size

    def forward(self, numerical_input, image_input):
        """
        Forward pass: Transforms input values into latent representation and predicts image features.

        Args:
            numerical_input (torch.Tensor): Numerical inputs of shape (batch_size, numerical_input_size).
            image_input (torch.Tensor): Image inputs of shape (batch_size, img_size, img_size).

        Returns:
            torch.Tensor: Predicted features of shape (batch_size, feature_output_size).
        """
        numerical_latent = self.numerical_encoder(numerical_input)
        image_latent = self.img_encoder(image_input.view(-1, image_input.size(-1) ** 2))  # Flatten image
        combined_latent = torch.cat([numerical_latent, image_latent], dim=1)
        predicted_features = self.feature_predictor(combined_latent)
        return predicted_features
    

    def fit(self, dataloader, epochs=10, lr=1e-3, device='cpu'):
            """
            Train the model using the provided dataloader.

            Args:
                dataloader (DataLoader): PyTorch DataLoader providing paired inputs (numerical, images, features).
                epochs (int): Number of training epochs.
                lr (float): Learning rate for optimizer.
                device (str): Device to train on ('cpu' or 'cuda').
            """
            self.to(device)
            optimizer = Adam(self.parameters(), lr=lr)
            criterion = nn.MSELoss()  # Loss for regression tasks

            for epoch in range(epochs):
                epoch_loss = 0
                self.train()
                for numerical_input, image_input, target_features in dataloader:
                    numerical_input = numerical_input.to(device)
                    image_input = image_input.to(device)
                    target_features = target_features.to(device)

                    # Forward pass
                    predicted_features = self(numerical_input, image_input)

                    # Compute loss
                    loss = criterion(predicted_features, target_features)

                    # Backward pass and optimization
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    epoch_loss += loss.item()

                print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataloader):.4f}")

# 4. Fit and train

In [272]:
model = MultiModalValueToImageFeaturePredict()

# Train the model with paired multi-modal data
model.fit(
    dataloader=(results,  # threshold, peak, trough
    train_ds),  
    epochs=10
)


ValueError: not enough values to unpack (expected 3, got 0)

5. Run optimizer to minimize loss